# Transfer Learning with Pretrained Models

## 1. Introduction
- **Transfer Learning** means reusing a model trained on a large dataset (like ImageNet) for a different but related task.
- Instead of training from scratch:
  - We keep the **learned features** (edges, shapes, textures, etc.).
  - Replace the final classification layer with a new one for our dataset.
- Advantages:
  - Faster training.
  - Better performance with less data.
  - Avoids overfitting on small datasets.

## 2. Workflow
1. Load a pretrained model (e.g., VGG16, ResNet, MobileNet).
2. Freeze some layers to keep existing weights.
3. Add new layers on top for the new dataset.
4. Train only the new layers (or fine-tune the whole model later).

## 3. Example: Using VGG16 on CIFAR-10

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)

# Load pretrained VGG16 model (without top layer)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze layers
for layer in base_model.layers:
    layer.trainable = False

# Build new model
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

# Evaluate
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc:.2f}")

## 4. Key Notes
- We froze the pretrained layers so they don’t get modified.
- Only the new layers were trained.
- For better results, we can unfreeze some deeper layers and fine-tune them.
- Transfer learning is widely used in **image classification, NLP, and speech recognition**.